In [2]:
import pandas as pd
import re
from dateutil.parser import parse
import joblib
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import json
import googlemaps
from datetime import datetime
from dateutil.parser import parse

In [3]:
df = pd.read_csv('DemoData.csv')

In [4]:
# 從地址中提取zipcode
c=0
zipcodeList = []
for i in df['address']:
    zipcode = re.search('WA 98[0-9]+', i).group().replace('WA ', '')
    zipcodeList.append(zipcode)
df['zipcode'] = zipcodeList
df['zipcode'] = df['zipcode'].astype(int)

In [5]:
# 外部資料結合
outsideCh = pd.read_excel('外部資料.xlsx', sheet_name='中文')
outsideEn = pd.read_excel('外部資料.xlsx', sheet_name='英文')
exchangeRate = pd.read_csv('exchangeRate.csv')
taxRate = pd.read_csv('taxrate.csv')
outsideCh.drop(['Unnamed: 23', 'Unnamed: 24'],axis=1, inplace=True)
taxRate.columns = ['zipcode', 'salesTaxRate']
exchangeRate.columns = ['date','ExchangeRate']

In [6]:
# 日期統一格式
for i in range(len(df)):
    df.iloc[i, 3] = datetime.strftime(parse(df.iloc[i, 3]), '%m/%d/%Y')

In [7]:
# 資料合併
allData = pd.merge(left=df, right=outsideEn,
                   how='inner', on='zipcode')

allData = pd.merge(left=allData, right=outsideCh,
                   how='inner', on='zipcode')

allData = pd.merge(left=allData, right=exchangeRate,
                   how='left', on='date')

allData = pd.merge(left=allData, right=taxRate,
                   how='left', on='zipcode')

In [8]:
# date 改年份 , 刪掉propertytype
allData['date'] = allData['date'].str.replace('[0-9]{1,2}/[0-9]{1,2}/', '').astype(int)
allData.drop(['PropertyType'], axis=1, inplace=True)

<ipython-input-8-b6c2b2431ebe>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  allData['date'] = allData['date'].str.replace('[0-9]{1,2}/[0-9]{1,2}/', '').astype(int)


In [9]:
# walk, transit 合併進去
traffic = pd.read_csv('trafficScore.csv')
allData = pd.merge(left=allData,
                   right=traffic,
                   how='inner',
                   on='address')

In [10]:
allData = allData.reindex(columns=['address', 'date', 'zipcode', 'bed', 'bath', 'sqft', 'walk', 'transit', 'school1', 'school2',
                                   'school3', 'yearBulit', 'lot', 'pt', 'ta',
                                   'population density', 'housing units ', 'land area',
                                   'median household income ', 'households with kids ',
                                   'year housing was built before 1970',
                                   'year housing was built after 1970', 'household income < 25k',
                                   'household income 25k~60K', 'hosuehold income 60k~150k',
                                   'household income > 150k', '~25k', '25k~60k', '60k~150k', '150k~',
                                   'means of transportation to work (Car)',
                                   'educational attainment (~high)', 'associate', 'bachelor', 'master',
                                   '人口數', '性別男', '性別女', '性別比', '白人', '黑人', '西班牙', '亞洲', '夏威夷', '印度裔', '其他',
                                   '白人比率', '黑人.1', '西班牙裔', '亞洲.1', '印度裔.1', '其他.1', '年齡中位數', '男性年齡中位數',
                                    '女性年齡中位數', '每戶人數', '每戶平均收入', 'ExchangeRate', 'salesTaxRate', 'price'])

In [11]:
allData.columns = [ 'address', 'soldout', 'zipcode', 'bedroom', 'bathroom', 'sqft',
       'walkscore', 'transitscore', 'grade1_5', 'grade6_8', 'grade9_12',
       'houseyear', 'lot', 'propertyTax', 'totalAssessment',
       'populationDensity', 'housingUnits ', 'landArea', 'medianIncome ',
       'withKids ', 'before_1970', 'after_1970', 'income_25k', 'income25k_60K',
       'income60k_150k', 'income150k_', '_25k', '25k_60k', '60k_150k', '150k_',
       'CommuteByCar_mean', 'highschool', 'associate', 'bachelor', 'master',
       'population', 'male', 'female', 'sexRatio', 'White', 'African',
       'Latino', 'Asian', 'Hawaiian', 'Indian', 'Others', 'WhiteRatio',
       'AfricanRatio', 'LatinoRatio', 'AsianRatio', 'IndianRatio',
       'OthersRatio', 'MedianAge', 'MaleMedianAge', 'FemaleMedianAge',
       'People_house', 'AverageIncome_house', 'ExchangeRate', 'salesTaxRate', 'houseprice']

In [12]:
# 刪掉機器學習不需要的欄位
allData = allData.drop(columns = ['_25k', '25k_60k','60k_150k', '150k_',
                              'male', 'female', 'sexRatio',
                              'White', 'African', 'Asian', 
                              'Hawaiian', 'Indian', 'Others',
                              'WhiteRatio','MaleMedianAge',
                              'FemaleMedianAge'])

In [13]:
# 機器學習
xgb = joblib.load('xgboost_model.pkl')

/Users/louis/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/louis/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator Pipeline from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [14]:
# 限定範圍
allData = allData[allData['houseprice'] <= 3000000]
allData = allData[allData['houseprice'] >= 200000]
allData = allData[allData['lot'] < 100000]

In [15]:
allData['zipcode'] = allData['zipcode'].astype(str)
allData['zipcode'] = allData['zipcode'].astype('category')
X = allData.iloc[:, 1:-1].values
y = allData.iloc[:, -1].values
y_pred = xgb.predict(X)

In [16]:
allData['zipcode'].memory_usage(index=False)

419

In [17]:
r2_score(y ,y_pred)

0.8924714793472027

In [20]:
#增加預測欄位 便罷價錢欄位單位改成 1 sqft 多少錢
pred_price = [y_pred[i] /  X[i][4] for i in range(len(X))]                                                  
allData['Predict_price_sqft'] = pred_price
allData['Actual_price_sqft'] = allData['houseprice'] / ['sqft']

In [20]:
# 只留小數後兩位
allDara = allData.round(2)

In [22]:
# googleapi 金鑰
gmaps = googlemaps.Client(key='google_api_key')

In [23]:
# 取得經緯度
lis = []
for address in new['address']:
    l = []
    a = gmaps.geocode(address= address)
    lat = a[0]['geometry']['location']['lat']
    lng = a[0]['geometry']['location']['lng']
    l.append(address)
    l.append(lat)
    l.append(lng)
    lis.append(l)

In [24]:
latLng = pd.DataFrame(lis, columns=['address','lat', 'lng'])

In [25]:
df1 = pd.merge(left=new,
         right=latLng,
         how='inner',
         on ='address')

In [26]:
webData = df1[['zipcode', 'address', 'lat', 'lng', 'bedroom', 'bathroom', 'sqft',
               'Predict_price_sqft', 'Actual_price_sqft']]

In [35]:
webData

,zipcode,address,lat,lng,bedroom,bathroom,sqft,Predict_price_sqft,Actual_price_sqft
0,98010,"29630 232nd Ave Se, Black Diamond, WA 98010",47.335882,-122.031602,3.0,1.0,2983,297.09,301.37
1,98010,"32330 2nd Ave, Black Diamond, WA 98010",47.311454,-122.005999,4.0,1.0,1540,276.57,324.64
2,98010,"29620 232nd Ave Se, Black Diamond, WA 98010",47.336286,-122.032137,3.0,2.0,2020,405.00,363.86
3,98010,"21604 Se 290th Ct, Black Diamond, WA 98010",47.342229,-122.053489,3.0,1.0,1360,339.31,349.26
4,98010,"32330 3rd Ave, Black Diamond, WA 98010",47.311551,-122.003780,4.0,1.5,2420,223.14,227.27
...,...,...,...,...,...,...,...,...,...
148,98106,"9203 12th Ave Sw, Seattle, WA 98106",47.520853,-122.350995,3.0,2.0,1120,549.25,491.07
149,98106,"5657 23rd Ave Sw, Seattle, WA 98106",47.550269,-122.362073,5.0,3.0,2090,387.60,370.81
150,98106,"8101 12th Ave Sw, Seattle, WA 98106",47.529972,-122.350916,3.0,1.0,1120,568.31,579.29
151,98106,"5403 23rd Ave Sw, Seattle, WA 98106",47.553508,-122.361972,5.0,2.0,1370,421.02,419.71


In [36]:
webData.to_csv('webData.csv', index=0)